# TensorFlow Assignment: Multilayer Perceptron (MLP) and Convolutional Neural Network (CNN)

**[Duke Community Standard](http://integrity.duke.edu/standard.html): By typing your name below, you are certifying that you have adhered to the Duke Community Standard in completing this assignment.**

Name: Yunzhen Zou

Now that you've run through a simple logistic regression model on MNIST, let's see if we can do better (Hint: we can). For this assignment, you'll build a multilayer perceptron (MLP) and a convolutional neural network (CNN), two popular types of neural networks, and compare their performance. Some potentially useful code:

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# Import data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
# Helper functions for creating weight variables
def weight_variable(shape):
    """weight_variable generates a weight variable of a given shape."""
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    """bias_variable generates a bias variable of a given shape."""
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

# Tensorflow Function that might also be of interest:
# tf.nn.sigmoid()
# tf.nn.relu()

### Multilayer Perceptron

Build a multilayer perceptron for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> fully connected (500 hidden units) -> nonlinearity (Sigmoid/ReLU) -> fully connected (10 hidden units) -> softmax

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

In [5]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32, [None, 10])### MNIST labels enter graph here ###

# Define the graph
y1 = tf.placeholder(tf.float32, [None, 500])
y_mlp = tf.placeholder(tf.float32, [None, 10])

### Create your MLP here##
W1 = weight_variable([784, 500])
W2 = weight_variable([500, 10])
b1 = bias_variable([500])
b2 = bias_variable([10])
y1 = tf.matmul(x, W1) + b1
y1 = tf.nn.sigmoid(y1)# or y1 = tf.nn.relu(y1)
y_mlp = tf.matmul(y1, W2) + b2
y_mlp = tf.nn.softmax(y_mlp)
### Make sure to name your MLP output as y_mlp ###



# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_mlp))

# Optimizer
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_mlp, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(4000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(100)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

step 0, validation accuracy 0.103
step 250, validation accuracy 0.653
step 500, validation accuracy 0.784
step 750, validation accuracy 0.805
step 1000, validation accuracy 0.787
step 1250, validation accuracy 0.818
step 1500, validation accuracy 0.82
step 1750, validation accuracy 0.829
step 2000, validation accuracy 0.835
step 2250, validation accuracy 0.834
step 2500, validation accuracy 0.858
step 2750, validation accuracy 0.837
step 3000, validation accuracy 0.85
step 3250, validation accuracy 0.831
step 3500, validation accuracy 0.835
step 3750, validation accuracy 0.849
test accuracy 0.8358


#### Comparison

How do the sigmoid and rectified linear unit (ReLU) compare?

***

[Your answer here]
Rectified linear unit performed better compared to sigmiod function.In my experiment, I can get test accuracy of 96% when ReLU serves as activation function but only 83% or so when use sigmoid function.

***

### Convolutional Neural Network

Build a simple 2-layer CNN for MNIST digit classfication. Feel free to play around with the model architecture and see how the training time/performance changes, but to begin, try the following:

Image -> CNN (32 5x5 filters) -> nonlinearity (ReLU) ->  (2x2 max pool) -> CNN (64 5x5 filters) -> nonlinearity (ReLU) -> (2x2 max pool) -> fully connected (1024 hidden units) -> nonlinearity (ReLU) -> fully connected (10 hidden units) -> softmax

Some additional functions that you might find helpful:

In [ ]:
# Convolutional neural network functions
def conv2d(x, W):
    """conv2d returns a 2d convolution layer with full stride."""
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    """max_pool_2x2 downsamples a feature map by 2X."""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Tensorflow Function that might also be of interest:
# tf.reshape()

Skeleton framework for you to fill in (Code you need to provide is marked by `###`):

*Hint: Convolutional Neural Networks are spatial models. You'll want to transform the flattened MNIST vectors into images for the CNN. Similarly, you might want to flatten it again sometime before you do a softmax. You also might want to look into the  [conv2d() documentation](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d) to see what shape kernel/filter it's expecting.*

In [ ]:
# Model Inputs
x = tf.placeholder(tf.float32, [None, 784])### MNIST images enter graph here ###
y_ = tf.placeholder(tf.float32, [None, 10])### MNIST labels enter graph here ###
x_image = tf.reshape(x,[-1,28,28,1])
# Define the graph
#y_conv1 = tf.placeholder(tf.float32, [None, 784])
#conv1
W_conv1 = weight_variable([5, 5, 1, 32]) 
b_conv1 = bias_variable([32])
y_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
y_pool1 = max_pool_2x2(y_conv1)

#conv2
W_conv2 = weight_variable([5,5,32,64])
b_conv2 = bias_variable([64])
y_conv2 = tf.nn.relu(conv2d(y_pool1,W_conv2)+b_conv2)
y_pool2 = max_pool_2x2(y_conv2)

#fc1
y_flatten = tf.reshape(y_pool2,[-1,7*7*64])
W_fc1 = weight_variable([7*7*64,1024])
b_fc1 = bias_variable([1024])
y_fc1 = tf.nn.relu(tf.matmul(y_flatten,W_fc1)+b_fc1)
#fc2
W_fc2 = weight_variable([1024,10])
b_fc2 = bias_variable([10])
y_conv = tf.nn.softmax(tf.matmul(y_fc1,W_fc2)+b_fc2)

### Create your CNN here##
### Make sure to name your CNN output as y_conv ###
# print(y_conv.shape)


# Loss 
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))

# Optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# Evaluation
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Training regimen
    for i in range(10000):
        # Validate every 250th batch
        if i % 250 == 0:
            validation_accuracy = 0
            for v in range(10):
                batch = mnist.validation.next_batch(50)
#                 print(batch[0].shape)
#                 print(batch[1].shape)
                validation_accuracy += (1/10) * accuracy.eval(feed_dict={x: batch[0], y_: batch[1]})
            print('step %d, validation accuracy %g' % (i, validation_accuracy))
        
        # Train    
        batch = mnist.train.next_batch(50)
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})

    print('test accuracy %g' % accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

(?, 10)
step 0, validation accuracy 0.132
step 250, validation accuracy 0.678
step 500, validation accuracy 0.71
step 750, validation accuracy 0.748
step 1000, validation accuracy 0.774
step 1250, validation accuracy 0.78
step 1500, validation accuracy 0.778
step 1750, validation accuracy 0.802
step 2000, validation accuracy 0.806
step 2250, validation accuracy 0.814
step 2500, validation accuracy 0.81
step 2750, validation accuracy 0.788
step 3000, validation accuracy 0.826
step 3250, validation accuracy 0.788
step 3500, validation accuracy 0.764
step 3750, validation accuracy 0.97
step 4000, validation accuracy 0.984
step 4250, validation accuracy 0.974
step 4500, validation accuracy 0.978
step 4750, validation accuracy 0.974
step 5000, validation accuracy 0.98
step 5250, validation accuracy 0.972
step 5500, validation accuracy 0.984
step 5750, validation accuracy 0.99
step 6000, validation accuracy 0.99
step 6250, validation accuracy 0.992
step 6500, validation accuracy 0.986
step 6

Some differences from the logistic regression model to note:

- The CNN model might take a while to train. Depending on your machine, you might expect this to take up to half an hour. If you see your validation performance start to plateau, you can kill the training.

- The logistic regression model we used previously was pretty basic, and as such, we were able to get away with using the GradientDescentOptimizer, which performs implements the gradient descent algorithm. For more difficult optimization spaces (such as the ones deep networks pose), we might want to use more sophisticated algorithms. Prof David Carlson has a lecture on this later.
    
- Because of the larger size of our network, notice that our minibatch size has shrunk.
    
- We've added a validation step every 250 minibatches. This let's us see how our model is doing during the training process, rather than sit around twiddling our thumbs and hoping for the best when training finishes. This becomes especially significant as training regimens start approaching days and weeks in length. Normally, we validate on the entire validation set, but for the sake of time we'll just stick to 10 validation minibatches (500 images) for this homework assignment.

#### Comparison

How do the MLP and CNN compare in accuracy? Training time? Why would you use one vs the other? Is there a problem you see with MLPs when applied to other image datasets?

***

[Your answer here]
The accuracy achieved by CNN is over 99% while 96% by MLP. CNN performed slightly better than MLP but much more time-consuming for the training phase. MLP would be a better choice if given MNIST as the input data becaused of its simplicity and efficiency. However, with the increased complexity of other image datasets, only several fully connected layers in MLP is far from enough to learn complicated features from these data.Without convolutional layer,number of parameters in MLP will become incredibly large when adding extra layer to learn more complicated features. Besides, it's easy for the model to overfit with so many parameters. 

***